In [21]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.express as px
from vnpy_ctastrategy.backtesting import load_tick_data
from vnpy.trader.constant import Exchange, Status, Direction, Offset

In [22]:
df = pd.read_pickle('./bkt_res.pkl')
df.index = pd.DatetimeIndex(df.index)
df

,close_price,pre_close,trades,trade_count,start_pos,end_pos,turnover,commission,slippage,trading_pnl,holding_pnl,total_pnl,net_pnl
date,,,,,,,,,,,,,
2021-08-06,6898.2,1.0,"[TradeData(gateway_name='BACKTESTING', symbol=...",26,0,0,35760680.0,822.49564,0.0,-2280.0,0.0,-2280.0,-3102.49564
2021-08-09,6913.8,6898.2,"[TradeData(gateway_name='BACKTESTING', symbol=...",44,0,0,60456960.0,1390.51008,0.0,1680.0,0.0,1680.0,289.48992
2021-08-10,6963.6,6913.8,"[TradeData(gateway_name='BACKTESTING', symbol=...",26,0,0,36002520.0,828.05796,0.0,-1880.0,0.0,-1880.0,-2708.05796
2021-08-11,6996.2,6963.6,"[TradeData(gateway_name='BACKTESTING', symbol=...",10,0,0,13934600.0,320.49580,0.0,-280.0,0.0,-280.0,-600.49580
2021-08-12,7054.6,6996.2,"[TradeData(gateway_name='BACKTESTING', symbol=...",34,0,0,47666880.0,1096.33824,0.0,-2560.0,0.0,-2560.0,-3656.33824
2021-08-13,7080.0,7054.6,"[TradeData(gateway_name='BACKTESTING', symbol=...",26,0,0,36596880.0,841.72824,0.0,-480.0,0.0,-480.0,-1321.72824
2021-08-16,7045.4,7080.0,"[TradeData(gateway_name='BACKTESTING', symbol=...",38,0,0,53675920.0,1234.54616,0.0,-3920.0,-0.0,-3920.0,-5154.54616
2021-08-17,6894.6,7045.4,"[TradeData(gateway_name='BACKTESTING', symbol=...",16,0,0,22339760.0,513.81448,0.0,-1440.0,-0.0,-1440.0,-1953.81448
2021-08-18,6916.6,6894.6,"[TradeData(gateway_name='BACKTESTING', symbol=...",12,0,0,16541200.0,380.44760,0.0,-960.0,0.0,-960.0,-1340.44760


In [23]:
date = pd.to_datetime('2021-09-23')
trades = df.loc[date, 'trades']
trade_df = pd.DataFrame([(x.datetime, x.direction, x.offset, x.price) for x in trades],
             columns=['datetime', 'direction', 'offset', 'price'])

quotes=  load_tick_data(trades[0].symbol,
                        trades[0].exchange,
                        date,
                        date + datetime.timedelta(days=1)
                        )
price = pd.Series(
    [x.last_price for x in quotes],
    index=[x.datetime for x in quotes])
fig = px.line(price, title='intra-day trades', width=1000, height=800, log_y=True)

buy = trade_df[(trade_df['direction']==Direction.LONG)&(trade_df['offset']==Offset.OPEN)]
sell = trade_df[(trade_df['direction']==Direction.SHORT)&(trade_df['offset']==Offset.CLOSE)]
short = trade_df[(trade_df['direction']==Direction.SHORT)&(trade_df['offset']==Offset.OPEN)]
cover = trade_df[(trade_df['direction']==Direction.LONG)&(trade_df['offset']==Offset.CLOSE)]
fig.add_trace(go.Scatter(x=buy['datetime'], y=buy['price'], marker_symbol='triangle-up', mode="markers", marker_size=12, marker_color='red', name='开多'))
fig.add_trace(go.Scatter(x=sell['datetime'], y=sell['price'], marker_symbol='triangle-down', mode="markers", marker_size=12, marker_color='red', name='平多'))
fig.add_trace(go.Scatter(x=short['datetime'], y=short['price'], marker_symbol='triangle-down', mode="markers", marker_size=12, marker_color='green', name='开空'))
fig.add_trace(go.Scatter(x=cover['datetime'], y=cover['price'], marker_symbol='triangle-up', mode="markers", marker_size=12, marker_color='green', name='开空'))
fig.update_layout(legend=dict(orientation="h"))
fig.show()

In [24]:
df = pd.read_pickle('optimization_result.pkl')
df.sort_values('sharpe_ratio', ascending=False).style

,start_date,end_date,total_days,profit_days,loss_days,capital,end_balance,max_drawdown,max_ddpercent,max_drawdown_duration,total_net_pnl,daily_net_pnl,total_commission,daily_commission,total_slippage,daily_slippage,total_turnover,daily_turnover,total_trade_count,daily_trade_count,total_return,annual_return,daily_return,return_std,sharpe_ratio,return_drawdown_ratio,threshold,threshold_close,signal_df_path,exchange,fixed_size,close_time_list
0,2021-01-05,2021-08-03,141,0,141,1000000,-6606377.283240,-7569376.998760,-786.020689,210,-7606377.283240,-53945.938179,1754897.283240,12446.080023,0.000000,0.000000,76299881880.000000,541133914.042553,59206,419.900709,-760.637728,-1294.702516,1.312198,10.568712,1.923459,-0.967707,0.000100,-0.000180,F:\HX\HFT-ML\exp\playground\test_ic_catboost\y_hat_training.pkl,Exchange.CFFEX,1,"[(11, 30), (15, 50)]"
5,2021-01-05,2021-08-03,141,0,141,1000000,-3952094.926840,-4929731.835320,-504.249767,210,-4952094.926840,-35121.240616,1173654.926840,8323.793807,0.000000,0.000000,51028475080.000000,361904078.581560,39610,280.921986,-495.209493,-842.909775,2.762659,23.590692,1.814229,-0.982072,0.000110,0.000001,F:\HX\HFT-ML\exp\playground\test_ic_catboost\y_hat_training.pkl,Exchange.CFFEX,1,"[(11, 30), (15, 50)]"
8,2021-01-05,2021-08-03,141,0,141,1000000,-2504130.144960,-3489676.940360,-354.085362,210,-3504130.144960,-24851.986844,813730.144960,5771.135780,0.000000,0.000000,35379571520.000000,250918946.950355,27466,194.794326,-350.413014,-596.447684,2.347726,25.217466,1.442287,-0.989629,0.000120,0.000001,F:\HX\HFT-ML\exp\playground\test_ic_catboost\y_hat_training.pkl,Exchange.CFFEX,1,"[(11, 30), (15, 50)]"
14,2021-01-05,2021-08-03,141,0,141,1000000,-738333.228600,-1735451.655600,-174.046694,210,-1738333.228600,-12328.604458,389493.228600,2762.363323,0.000000,0.000000,16934488200.000000,120102753.191489,13150,93.262411,-173.833323,-295.886507,1.476050,16.190426,1.412369,-0.998774,0.000140,0.000001,F:\HX\HFT-ML\exp\playground\test_ic_catboost\y_hat_training.pkl,Exchange.CFFEX,1,"[(11, 30), (15, 50)]"
17,2021-01-05,2021-08-03,141,0,141,1000000,-185107.392280,-1184519.643960,-118.521625,210,-1185107.392280,-8405.016967,260507.392280,1847.570158,0.000000,0.000000,11326408360.000000,80329137.304965,8800,62.411348,-118.510739,-201.720407,1.106641,20.106768,0.852648,-0.999908,0.000150,0.000001,F:\HX\HFT-ML\exp\playground\test_ic_catboost\y_hat_training.pkl,Exchange.CFFEX,1,"[(11, 30), (15, 50)]"
15,2021-01-05,2021-08-03,141,0,141,1000000,-271469.093600,-1270321.328720,-127.178103,210,-1271469.093600,-9017.511302,265069.093600,1879.922650,0.000000,0.000000,11524743200.000000,81735767.375887,8954,63.503546,-127.146909,-216.420271,1.452724,28.088724,0.801229,-0.999755,0.000150,-0.000180,F:\HX\HFT-ML\exp\playground\test_ic_catboost\y_hat_training.pkl,Exchange.CFFEX,1,"[(11, 30), (15, 50)]"
9,2021-01-05,2021-08-03,141,0,141,1000000,-1713881.540440,-2699492.243760,-273.890313,210,-2713881.540440,-19247.386812,583681.540440,4139.585393,0.000000,0.000000,25377458280.000000,179981973.617021,19708,139.773050,-271.388154,-461.937283,1.192408,29.663764,0.622736,-0.990864,0.000130,-0.000180,F:\HX\HFT-ML\exp\playground\test_ic_catboost\y_hat_training.pkl,Exchange.CFFEX,1,"[(11, 30), (15, 50)]"
3,2021-01-05,2021-08-03,141,0,141,1000000,-4404190.712760,-5379608.246280,-551.518510,210,-5404190.712760,-38327.593708,1210070.712760,8582.061793,0.000000,0.000000,52611770120.000000,373133121.418440,40844,289.673759,-540.419071,-919.862249,-0.020296,22.795289,-0.013794,-0.979875,0.000110,-0.000180,F:\HX\HFT-ML\exp\playground\test_ic_catboost\y_hat_training.pkl,Exchange.CFFEX,1,"[(11, 30), (15, 50)]"
6,2021-01-05,2021-08-03,141,0,141,1000000,-2820305.317680,-3804992.747320,-386.416302,210,-3820305.317680,-27094.363955,837345.317680,5938.619274,0.000000,0.000000,36406318160.000000,258200838.014184,28272,200.510638,-382.030532,-650.264735,-0.333282,26.909156,-0.191875,-0.988650,0.000120,-0.000180,F:\HX\HFT-ML\exp\playground\test_ic_catboost\y